In [1]:
from train import *

arg_dict = {
        "env": "11_vs_11_easy_stochastic",
        "num_processes": 1,
        "batch_size": 4,   
        "buffer_size": 2,
        "rollout_len": 10,
        "lstm_size" : 196,
        "k_epoch" : 3,
        "summary_game_window" : 2,
        "model_save_interval" : 100,
        "learning_rate" : 0.0001,
        "gamma" : 0.992,
        "lmbda" : 0.96,
        "entropy_coef" : 0.0,
        "trained_model_dir" : "logs/[10-28]17.17.12/model_120.tar",   # default : None
#         "trained_model_dir" : None,   # default : None
        "print_mode" : False,
        
        "encoder" : "encoder_raw",
        "rewarder" : "rewarder_se",
        "model" : "ppo_conv1d"
    }
    

main(arg_dict)

Trained model suffessfully loaded
Learner process started
Actor process 0 started
step : 144 loss -0.026815947455664475 data_q 19 summary_q 0
warning. data remaining. queue size :  19
step : 168 loss 0.009455755508194367 data_q 11 summary_q 0
warning. data remaining. queue size :  11
step : 192 loss -0.006210696456643443 data_q 3 summary_q 0
step : 216 loss 0.006314237340120599 data_q 1 summary_q 0
step : 240 loss -0.015057630836963654 data_q 1 summary_q 0


Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/gfootball/football-paris/learner.py", line 119, in learner
    time.sleep(0.1)
KeyboardInterrupt


KeyboardInterrupt: 

In [3]:
prob = 1
a=1
m=1

assert prob != 0, 'prob 0 ERROR!!!! a : {}, m:{}'.format(a,m)

print('hi')

hi
